In [8]:
import numpy as np
import matplotlib

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, CheckButtons
from scipy import signal

### Функція для створення гармоніки з шумом

In [9]:
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise):
    t = np.linspace(0, 10, 500)
    omega = 2 * np.pi * frequency  # Кутова частота
    y = amplitude * np.sin(omega * t + phase)  # Гармонічна функція
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), t.shape)  # Шум
    y_noisy = y + noise
    
    # Очищення попереднього графіку
    ax.clear()
    
    # Візуалізація
    if show_noise:
        ax.plot(t, y_noisy, color='orange')
    ax.plot(t, y, color='blue', linestyle="--")
    ax.set_xlabel('Time')
    ax.set_ylabel('Amplitude')

    # Фільтрація за допомогою низькочастотного фільтра
    filtered_y = filter_signal(t, y_noisy, cutoff_frequency_slider.val)
    ax.plot(t, filtered_y, color='red')

    plt.draw()

### Функція для скидання параметрів

In [10]:
def reset_parameters(event):
    amplitude_slider.reset()
    frequency_slider.reset()
    phase_slider.reset()
    noise_mean_slider.reset()
    noise_covariance_slider.reset()
    show_noise_checkbox.set_active(0)


### Оновлення графіку при зміні значень слайдерів

In [11]:
def update(val):
    amplitude = amplitude_slider.val
    frequency = frequency_slider.val
    phase = phase_slider.val
    noise_mean = noise_mean_slider.val
    noise_covariance = noise_covariance_slider.val
    show_noise = show_noise_checkbox.get_status()[0]
    
    harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise)

### Функція для фільтрації сигналу

In [12]:
def filter_signal(t, y_noisy, cutoff_frequency):
    # Налаштування фільтра
    fs = 100  # Частота дискретизації
    nyquist = 0.5 * fs  # Частота Найквіста
    normal_cutoff = cutoff_frequency / nyquist  # Нормалізована частота зрізу
    b, a = signal.butter(4, normal_cutoff, btype='low')  # 4-й порядок фільтра
    y_filtered = signal.filtfilt(b, a, y_noisy)  # Фільтрація сигналу
    return y_filtered

### Створення додатку

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
plt.subplots_adjust(left=0.1, bottom=0.5)

# Створення слайдерів
amplitude_slider = Slider(plt.axes([0.1, 0.29, 0.65, 0.03]), 'Amplitude', 0, 2, valinit=0.97, valstep=0.01)
frequency_slider = Slider(plt.axes([0.1, 0.25, 0.65, 0.03]), 'Frequency', 0.1, 3, valinit=0.267, valstep=0.01)
phase_slider = Slider(plt.axes([0.1, 0.21, 0.65, 0.03]), 'Phase', -np.pi, np.pi, valinit=0.000, valstep=0.01)
noise_mean_slider = Slider(plt.axes([0.1, 0.17, 0.65, 0.03]), 'Noise Mean', -2.0, 2.0, valinit=0.108, valstep=0.01)
noise_covariance_slider = Slider(plt.axes([0.1, 0.13, 0.65, 0.03]), 'Noise Covariance', 0.01, 2.0, valinit=0.101, valstep=0.01)
cutoff_frequency_slider = Slider(plt.axes([0.1, 0.09, 0.65, 0.03]), 'Cutoff Frequency', 0.1, 10.0, valinit=5.0, valstep=0.01)

# Кнопка reset
reset_button = Button(plt.axes([0.1, 0.02, 0.1, 0.05]), 'Reset')
reset_button.on_clicked(reset_parameters)

# Чекбокс для відображення шуму
show_noise_ax = plt.axes([0.75, 0.02, 0.15, 0.05])
show_noise_checkbox = CheckButtons(show_noise_ax, ['Показати шум'], [True])

# Підключення функції до слайдерів
amplitude_slider.on_changed(update)
frequency_slider.on_changed(update)
phase_slider.on_changed(update)
noise_mean_slider.on_changed(update)
noise_covariance_slider.on_changed(update)
cutoff_frequency_slider.on_changed(update)
show_noise_checkbox.on_clicked(lambda label: update(None))

# Початкове відображення графіка
harmonic_with_noise(amplitude_slider.val, frequency_slider.val, phase_slider.val, 
                    noise_mean_slider.val, noise_covariance_slider.val, show_noise_checkbox.get_status()[0])

plt.show()